In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Let's take a first look at the data.

In [2]:
df = pd.read_csv('data/311_test.csv')
df.head(5)

,Unique Key,Created Date,Closed Date,Agency,Agency Name,Complaint Type,Descriptor,Location Type,Incident Zip,Incident Address,...,Bridge Highway Name,Bridge Highway Direction,Road Ramp,Bridge Highway Segment,Garage Lot Name,Ferry Direction,Ferry Terminal Name,Latitude,Longitude,Location
0,36335488,06/01/2017 12:00:00 AM,06/23/2017 12:00:00 AM,DOHMH,Department of Health and Mental Hygiene,Rodent,Rat Sighting,1-2 Family Dwelling,10474,624 COSTER STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.812797,-73.886981,"(40.81279663744078, -73.8869808964065)"
1,36334571,06/01/2017 12:00:00 AM,06/22/2017 12:00:00 AM,DOHMH,Department of Health and Mental Hygiene,Rodent,Rat Sighting,Vacant Lot,11215,224 12 STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.668242,-73.989604,"(40.66824162997938, -73.98960394471317)"
2,36334446,06/01/2017 12:00:00 AM,06/21/2017 12:00:00 AM,DOHMH,Department of Health and Mental Hygiene,Unsanitary Pigeon Condition,Pigeon Waste,Commercial Building,10303,2501 FOREST AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.626336,-74.174961,"(40.62633610844385, -74.17496135020113)"
3,36332405,06/01/2017 12:00:00 AM,06/22/2017 12:00:00 AM,DOHMH,Department of Health and Mental Hygiene,Rodent,Rat Sighting,Vacant Lot,11215,224 12 STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.668242,-73.989604,"(40.66824162997938, -73.98960394471317)"
4,36332388,06/01/2017 12:00:00 AM,06/21/2017 12:00:00 AM,DOHMH,Department of Health and Mental Hygiene,Rodent,Mouse Sighting,3+ Family Apt. Building,11105,21-24 31 STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.777379,-73.909192,"(40.777379055532506, -73.9091923175842)"


# We can see the data is full of missing values. Let's clean it up!

In [ ]:
df.isnull().sum()

In [3]:
delete_columns = ["Intersection Street 1", "Intersection Street 2", "Address Type", "Landmark", "Facility Type",\
                  "Status", "X Coordinate (State Plane)", "Y Coordinate (State Plane)", "Park Facility Name",\
                  "Park Borough", "School Name", "School Number", "School Region", "School Code",\
                  "School Phone Number", 
                  "School Address", "School City", "School State", "School Zip", "School Not Found",\
                  "School or Citywide Complaint", "Vehicle Type", "Taxi Company Borough", "Taxi Pick Up Location",\
                  "Bridge Highway Name", "Bridge Highway Direction", "Road Ramp", "Bridge Highway Segment",\
                  "Garage Lot Name", "Ferry Direction", "Ferry Terminal Name","Due Date", \
                  "Resolution Action Updated Date","Incident Address","Street Name","Cross Street 1","Cross Street 2",\
                  "Location Type","Resolution Description"]

df.drop(delete_columns, axis=1, inplace=True)
df.shape

(14812, 14)

In [5]:
#Drop NAs in several categories and print new null sums
df = df.dropna(subset=['Latitude',"Longitude","Complaint Type", "Incident Zip"], how='any')
df.isnull().sum()

Unique Key            0
Created Date          0
Closed Date        1215
Agency                0
Agency Name           0
Complaint Type        0
Descriptor          167
Incident Zip          0
City                  1
Community Board       0
Borough               0
Latitude              0
Longitude             0
Location              0
dtype: int64

In [6]:
#Change incident_zip column datatype from float to str
#to map with GeoJSON
df['Incident Zip'] = df['Incident Zip'].astype(int)
df['Incident Zip'] = df['Incident Zip'].astype(str)
df['Incident Zip'].dtype

dtype('O')

# Let's clean the column name syntax as well. Making queries will become easier.

In [7]:
#Cleaning column name syntax
df.columns = ['unique_key','created_date', 'closed_date', 'agency', 'agency_name','complaint_type',\
                    'descriptor', 'incident_zip', 'city','community_board', 'borough', 'latitude', 'longitude',\
                    'location']
df.head(5)

,unique_key,created_date,closed_date,agency,agency_name,complaint_type,descriptor,incident_zip,city,community_board,borough,latitude,longitude,location
0,36335488,06/01/2017 12:00:00 AM,06/23/2017 12:00:00 AM,DOHMH,Department of Health and Mental Hygiene,Rodent,Rat Sighting,10474,BRONX,02 BRONX,BRONX,40.812797,-73.886981,"(40.81279663744078, -73.8869808964065)"
1,36334571,06/01/2017 12:00:00 AM,06/22/2017 12:00:00 AM,DOHMH,Department of Health and Mental Hygiene,Rodent,Rat Sighting,11215,BROOKLYN,06 BROOKLYN,BROOKLYN,40.668242,-73.989604,"(40.66824162997938, -73.98960394471317)"
2,36334446,06/01/2017 12:00:00 AM,06/21/2017 12:00:00 AM,DOHMH,Department of Health and Mental Hygiene,Unsanitary Pigeon Condition,Pigeon Waste,10303,STATEN ISLAND,01 STATEN ISLAND,STATEN ISLAND,40.626336,-74.174961,"(40.62633610844385, -74.17496135020113)"
3,36332405,06/01/2017 12:00:00 AM,06/22/2017 12:00:00 AM,DOHMH,Department of Health and Mental Hygiene,Rodent,Rat Sighting,11215,BROOKLYN,06 BROOKLYN,BROOKLYN,40.668242,-73.989604,"(40.66824162997938, -73.98960394471317)"
4,36332388,06/01/2017 12:00:00 AM,06/21/2017 12:00:00 AM,DOHMH,Department of Health and Mental Hygiene,Rodent,Mouse Sighting,11105,Astoria,01 QUEENS,QUEENS,40.777379,-73.909192,"(40.777379055532506, -73.9091923175842)"


In [8]:
df.to_csv('data/cleaned_311_test_2015.csv')